
<br>
<font>
<div dir=ltr align=center>
<img src="https://cdn.freebiesupply.com/logos/large/2x/sharif-logo-png-transparent.png" width=150 height=150> <br>
<font color=0F5298 size=7>
Artificial Intelligence <br>
<font color=2565AE size=5>
Computer Engineering Department <br>
Spring 2024<br>
<font color=3C99D size=5>
Practical Assignment 1 - Constraint Satisfaction Problem <br>
<font color=696880 size=4>
Amirabbas Afzali

____

# Personal Data

In [ ]:
# Set your student number
student_number = '99105429'
Name = 'Amirhossein'
Last_Name = 'Khoshbakht'

# Rules
- Make sure that all of your cells can be run perfectly.

---

# **Graph coloring**

Hello, In this question, we have an engaging problem scenario based on **graph coloring**. Imagine a fictional scenario involving a group of interstellar travelers exploring a mysterious alien planet called **Zyra**. The planet is covered in exotic flora and fauna, and the explorers need to map its regions while adhering to certain rules. Here’s the scenario:


## **The Zyran Flora Expedition**

### **Background:**

The Zyran Flora Expedition, led by renowned botanist Dr. Elara, has landed on the lush and vibrant planet Zyra. Their mission is to catalog the various plant species across the planet’s diverse ecosystems. However, there’s a catch: each plant species emits a unique energy signature that affects the surrounding vegetation. Dr. Elara wants to create a map of Zyra’s regions, ensuring that no two adjacent regions contain the same type of plant.

### **Problem Statement:**
Given the map of Zyra, represented as an undirected graph where each vertex corresponds to a region, design an algorithm to color the regions using as few colors as possible while adhering to the following rules:

1. **Color Constraints:**
- Each region can be assigned one of the available colors.
- No two adjacent regions (sharing a border) can have the same color if they contain the same type of plant.

2. **Objective:**
- Minimize the total number of colors used while ensuring that the map remains scientifically accurate.

3. **Input:**
- A graph representing Zyra’s regions, where vertices represent different ecological zones, and edges connect neighboring zones.

4. **Output:**
- A valid coloring scheme for the regions, using the fewest possible colors.

### **Example:**
Consider a simplified map of Zyra with the following regions and their connections:

**Regions:** A, B, C, D, E, F
**Connections:**
- A and B share a border.
- B and C share a border.
- C and D share a border.
- D and E share a border.
- E and F share a border.

The goal is to color these regions using as few colors as possible. One valid coloring could be:

- A: Green (contains unique plant species X)
- B: Blue (contains unique plant species Y)
- C: Red (contains unique plant species Z)
- D: Green (contains unique plant species X)
- E: Blue (contains unique plant species Y)
- F: Red (contains unique plant species Z)

In this example, only three colors (Green, Blue, and Red) are needed to represent Zyra’s diverse flora.


## Input
  - The first line determines $N$ (count of regions).
  - The third line determines $M$ (count of possible colors for each region).
  - in each of the following lines, a pair of region numbers (adjacent constraints) is given.

$$1 \le N \le 60$$
$$1 \le M \le 5$$

### Sample Input
This sample describes the previous example.

```
N = 6
# Colors
colors = 3
# Graph:
0,1
1,2
2,3
3,4
4,5
```

### Imports

Feel free to import any library you need.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import copy

np.random.seed(1234)

In [ ]:
# Global Variable variables
neighbours = {}
start_variables = {}

Necessary Functions and classes :

In [ ]:
##############################################
# no need to do any thing,
# However, you can change it if needed
##############################################

# Function to read data from the input file
def fileReader(textfile):
    neighbours.clear()
    start_variables.clear()
    with open(textfile, "r") as f:
        data = f.readlines()
    # Process each line of data
        for line in data:
            if line.startswith('#') or line.strip() == "":
                continue
            elif line.startswith('c'):
                colors = int(''.join(filter(str.isdigit, line)))
                continue
            # Split the line into two nodes
            node1, node2 = map(int, line.strip().split(","))
            # Update the neighbours
            if node1 not in neighbours:
                neighbours[node1] = []
            neighbours[node1].append(node2)
            if node2 not in neighbours:
                neighbours[node2] = []
            neighbours[node2].append(node1)
            # Update the variables
            if node1 not in start_variables:
                start_variables[node1] = [None, list(range(1, colors+1))]
            if node2 not in start_variables:
                start_variables[node2] = [None, list(range(1, colors+1))]


# Class definition of a state
class color:

    # Constructor to intialize the fields of the current state
    def __init__(self, assigned, variables):
        self.assigned = assigned   # Stores the variables that already have been assigned
        self.variables = variables # Stores the current state of the graph
        self.curr = None           # Stores the current variable chosen to assign from the MRV Heuristic

    # Function to print the variables with their values
    def printvar(self):
        for var in self.variables:
            print(f"Region ID : {var} - Region Color: {self.variables[var][0]}")


    # Function to visualise the final coloured graph
    def colorGraph(self ,report=True):
        G = nx.Graph()
        for node, neighbours_list in neighbours.items():
            G.add_node(node)
            for neighbour in neighbours_list:
                G.add_edge(node, neighbour)
        # Draw the graph
        pos = nx.spring_layout(G)
        # Define a color map
        colors_map = {1: 'red', 2: 'green', 3: 'blue', 4: 'yellow', 5: 'pink', 6: ' brown'}
        # Loop over the nodes and their assigned colors
        for node, (color, _) in self.variables.items():
            if color is not None:
                nx.set_node_attributes(G, {node: colors_map[color]}, 'color')
        # Draw the graph with node colors
        node_colors = nx.get_node_attributes(G, 'color')
        nx.draw(G, pos, with_labels=True, node_color=list(node_colors.values()))
        if  report:
            plt.show()

## Note
  - You _must_ implement and use AC-3 algorithm.
  - You _must_ implement and use the heuristic algorithms **MRV** and **LCV** in order to pass all tests in an appropriate time.

## Your code
**Note:** It's OK to change the signature for the given functions and the given structure is just a suggestion to help you with the implementation. (you can't remove or add any cells)

In [16]:
# Function to ensure arc consistency
def arc_consistency(variables):
    queue = [(x, y) for x in variables for y in variables if x != y]
    while queue:
        (xi, xj) = queue.pop(0)
        if remove_inconsistent_values(xi, xj, variables):
            for xk in variables:
                if xk != xi and (xk, xi) not in queue:
                    queue.append((xk, xi))
    return variables


# Function to remove the inconsistent values in an arc.

def remove_inconsistent_values(xi, xj, variables):
    removed = False
    for x in variables[xi][1]:
        if all(not constraint(x, y) for y in variables[xj][1]):
            variables[xi][1].remove(x)
            removed = True
    return removed

def constraint(x, y):
    return x != y

# Function to find the minimum remaining variable to be assigned
def minimum_remaining_variable(state):
    return sorted(state.variables.keys(), key=lambda var: (len(state.variables[var][1]), var))



# Function to find the least constraining value
def least_constraining_value(state, var):
    return sorted(state.variables[var][1], key=lambda val: sum(1 for neighbor in neighbours[var] if val in state.variables[neighbor][1]))



# Function to call the backtracking search
def backtrack(filename, report=True):
    fileReader(filename)
    state = color({}, copy.deepcopy(start_variables))
    solution = backtrack_search(state)
    if solution:
        solution.printvar()
        if report:
            solution.colorGraph()
    else:
        print("No solution exists")



# Recursive function that performs backtracking search based on the heuristics
def backtrack_search(state):
    stack = [state]
    while stack:
        current_state = stack.pop()
        if all(current_state.variables[var][0] is not None for var in current_state.variables):
            return current_state
        var = minimum_remaining_variable(current_state)[0]
        for value in least_constraining_value(current_state, var):
            new_state = color(copy.deepcopy(current_state.assigned), copy.deepcopy(current_state.variables))
            new_state.variables[var][0] = value
            new_state.variables[var][1] = [value]
            if arc_consistency(new_state.variables):
                stack.append(new_state)
    return None


## Validate:

In order to validate your implementation , run below blocks.

**Note that** correct implementation of different functions (LCV, MVR, ...) and final results are needed to get full score.


In [ ]:
if __name__ == "__main__":
    num_tests = 8
    for i in range(num_tests):
        print(f'Test {i+1}: ')
        backtrack(f'/content/data/input{i}.txt')
        print('='*90)


Test 1: 


### Plot results of first 6 test-cases :

In [ ]:
plt.figure(figsize=(15, 10))
idx = [331+i for i in range(6)]

for i in range(6):
    plt.subplot(idx[i])
    backtrack(f'/content/data/input{i}.txt',report=False)
    plt.title(f'Graph {i+1}')

plt.tight_layout()
plt.show()

# **Best regards :)**